# Fisher Class

In [1]:
import numpy as np
import sympy as smp
import matplotlib.pyplot as plt
import pandas as pd

# For interpolation
from scipy.interpolate import RectBivariateSpline, interp2d

import warnings

# Ignore DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message=".*divmax.*")

import logging
# Basic registry settings
logging.basicConfig(level=logging.INFO)

In [2]:
import Cosmo_util as cu
import Cosmo_integration as ci
import Cosmo_shear as cs

Note: redshifts have been re-sorted (earliest first)
Interpolation of matter power spectrum created.
Note: redshifts have been re-sorted (earliest first)
Interpolation of derivative of matter power spectrum with respect to k created.
Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)
Interpolation of derivative of matter power spectrum with respect to h created.
Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)
Interpolation of derivative of matter power spectrum with respect to Omega_m0 created.
Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)
Note: redshifts 

In [3]:
# Parametros fiduciales

Omega_b0_fid = 0.05
Omega_m0_fid = 0.32
h_fid = 0.67
ns_fid = 0.96
sigma8_fid = 0.816
Omega_DE0_fid = 0.68
w0_fid = -1.0
wa_fid = 0.0
gamma_fid = 0.55

c = 300000
Aia = 1.72
Cia = 0.0134

In [4]:
class Fisher:
    '''
    Calculate Fisher Matrix
    '''
    def __init__(self, params):
        self.num = params['num_params']
        self.universes = params['type']
        self.model = params['model']

    def trace(self, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, Cia):
        F = np.zeros((self.num + 2, self.num + 2))
        L_array = np.log10(np.logspace(np.log10(10), np.log10(1500), 100))
        cosmic_params = {'l': L_array , 'z': np.linspace(0.001, 2.5, 50), 'type': self.universes, 'model': self.model}
        A = cs.CosmicShear(cosmic_params)
        
        f_sky = 0.3636
        epsilon = 0.01
        C = np.zeros((10, 10), dtype=object)
        dC_dq_Om_m_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_h_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Om_b_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_ns_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_sig_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Aia_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Cia_matrix = np.zeros((10, 10), dtype=object)
        
        for i in range(10):
            for j in range(i, 10):
                logging.info(f"Calculated pair: {(i, j)}")
                C[i, j] = C[j, i] = A.Cosmic_Shear(i, j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, Cia)
                dC_dq_Om_m_matrix[i, j] = dC_dq_Om_m_matrix[j, i] = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, Cia, 'Omega_m0')
                dC_dq_h_matrix[i, j] = dC_dq_h_matrix[j, i] = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, Cia, 'h')
                dC_dq_Om_b_matrix[i, j] = dC_dq_Om_b_matrix[j, i] = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, Cia, 'Omega_b0')
                dC_dq_ns_matrix[i, j] =  dC_dq_ns_matrix[j, i]  = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, Cia, 'ns')
                dC_dq_sig_matrix[i, j] = dC_dq_sig_matrix[j, i] = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, Cia, 'sigma8')
                dC_dq_Aia_matrix[i, j] =  dC_dq_Aia_matrix[j, i]  = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, Cia, 'Aia')
                dC_dq_Cia_matrix[i, j] =  dC_dq_Cia_matrix[j, i]  = A.Der_C_parametro(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, Aia, Cia, 'Cia')

                
        C_matrix = np.array(C.tolist(), dtype=float)
        C_matrix_2d = np.squeeze(C_matrix)

        dC_dq_Om_m_matrix = np.array(dC_dq_Om_m_matrix.tolist(), dtype=float)
        dC_dq_h_matrix = np.array(dC_dq_h_matrix.tolist(), dtype=float)
        dC_dq_Om_b_matrix = np.array(dC_dq_Om_b_matrix.tolist(), dtype=float)
        dC_dq_ns_matrix = np.array(dC_dq_ns_matrix.tolist(), dtype=float)
        dC_dq_sig_matrix = np.array(dC_dq_sig_matrix.tolist(), dtype=float)
        dC_dq_Aia_matrix = np.array(dC_dq_Aia_matrix.tolist(), dtype=float)
        dC_dq_Cia_matrix = np.array(dC_dq_Cia_matrix.tolist(), dtype=float)

        dC_dq_Om_m_matrix_2d = np.squeeze(dC_dq_Om_m_matrix)
        dC_dq_h_matrix_2d = np.squeeze(dC_dq_h_matrix)
        dC_dq_Om_b_matrix_2d = np.squeeze(dC_dq_Om_b_matrix)
        dC_dq_ns_matrix_2d = np.squeeze(dC_dq_ns_matrix)
        dC_dq_sig_matrix_2d = np.squeeze(dC_dq_sig_matrix)
        dC_dq_Aia_matrix_2d = np.squeeze(dC_dq_Aia_matrix)
        dC_dq_Cia_matrix_2d = np.squeeze(dC_dq_Cia_matrix)
        
        for i, l in enumerate(L_array):

            if i < len(L_array) - 1:
                delta_l = 10**L_array[i+1] - 10**L_array[i]
            else: 
                 delta_l = 1
                 
            L_parameter = np.sqrt((2 / ((2*(10**l) + 1)*delta_l*f_sky)))
            C_inv = np.linalg.inv(L_parameter * C_matrix_2d[:, :, i])

            #OMEGA_M 
            Om_m_Om_m = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv)
            Om_m_h = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            Om_m_Om_b = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_m_ns = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_m_sig = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            Om_m_Aia = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Aia_matrix_2d[:, :, i] @ C_inv)
            Om_m_Cia = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Cia_matrix_2d[:, :, i] @ C_inv)
#

            #H
            h_h = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            h_Om_b = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            h_ns = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            h_sig = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            h_Aia = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Aia_matrix_2d[:, :, i] @ C_inv)
            h_Cia = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Cia_matrix_2d[:, :, i] @ C_inv)
#
            #OMEGA_B
            Om_b_Om_b = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_b_ns = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_b_sig = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            Om_b_Aia = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Aia_matrix_2d[:, :, i] @ C_inv)
            Om_b_Cia = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Cia_matrix_2d[:, :, i] @ C_inv)

            #NS
            ns_ns = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            ns_sig = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            ns_Aia = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_Aia_matrix_2d[:, :, i] @ C_inv)
            ns_Cia = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_Cia_matrix_2d[:, :, i] @ C_inv)

            #SIGMA8
            sig_sig = np.trace(dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv) 
            sig_Aia = np.trace(dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_Aia_matrix_2d[:, :, i] @ C_inv)  
            sig_Cia = np.trace(dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_Cia_matrix_2d[:, :, i] @ C_inv)

            #Aia
            Aia_Aia = np.trace(dC_dq_Aia_matrix_2d[:, :, i] @ C_inv @ dC_dq_Aia_matrix_2d[:, :, i] @ C_inv)  
            Aia_Cia = np.trace(dC_dq_Aia_matrix_2d[:, :, i] @ C_inv @ dC_dq_Cia_matrix_2d[:, :, i] @ C_inv)

            #Cia
            Cia_Cia = np.trace(dC_dq_Cia_matrix_2d[:, :, i] @ C_inv @ dC_dq_Cia_matrix_2d[:, :, i] @ C_inv)



            if self.model == 'ACDM_flat':
                coeficientes = {
                        (0, 0): Om_m_Om_m, (0, 1): Om_m_h, (0, 2): Om_m_Om_b, (0, 3): Om_m_ns, (0, 4): Om_m_sig, (0, 5): Om_m_Aia, (0, 6): Om_m_Cia,
                        (1, 1): h_h, (1, 2): h_Om_b, (1, 3): h_ns, (1, 4): h_sig, (1, 5): h_Aia, (1, 6): h_Cia,
                        (2, 2): Om_b_Om_b, (2, 3): Om_b_ns, (2, 4): Om_b_sig, (2, 5): Om_b_Aia, (2, 6): Om_b_Cia,
                        (3, 3): ns_ns, (3, 4): ns_sig, (3, 5): ns_Aia, (3, 6): ns_Cia,
                        (4, 4): sig_sig, (4, 5): sig_Aia, (4, 6): sig_Cia,
                        (5, 5): Aia_Aia, (5, 6): Aia_Cia,
                        (6, 6): Cia_Cia
                    }

                for (i, j), coef in coeficientes.items():
                    F[i, j] += coef

        Fisher = F + F.T - np.diag(F.diagonal())
        logging.info(f"Fisher Matrix: {Fisher}")

        return Fisher

In [5]:
params = {'num_params': 5, 'type': 'standard', 'model' : 'ACDM_flat'}

A = Fisher(params)

F = A.trace(Omega_m0_fid, h_fid, Omega_b0_fid, Omega_DE0_fid, w0_fid, wa_fid, ns_fid, sigma8_fid, gamma_fid, Aia, Cia)

Cov = np.linalg.inv(F)

print('Matriz de covarianzas')

print(' ')

print(Cov)

INFO:root:Calculated pair: (0, 0)
INFO:root:Calculated pair: (0, 1)
INFO:root:Calculated pair: (0, 2)
INFO:root:Calculated pair: (0, 3)
INFO:root:Calculated pair: (0, 4)
INFO:root:Calculated pair: (0, 5)
INFO:root:Calculated pair: (0, 6)
INFO:root:Calculated pair: (0, 7)
INFO:root:Calculated pair: (0, 8)
INFO:root:Calculated pair: (0, 9)
INFO:root:Calculated pair: (1, 1)
INFO:root:Calculated pair: (1, 2)
INFO:root:Calculated pair: (1, 3)
INFO:root:Calculated pair: (1, 4)
INFO:root:Calculated pair: (1, 5)
INFO:root:Calculated pair: (1, 6)
INFO:root:Calculated pair: (1, 7)
INFO:root:Calculated pair: (1, 8)
INFO:root:Calculated pair: (1, 9)
INFO:root:Calculated pair: (2, 2)
INFO:root:Calculated pair: (2, 3)
INFO:root:Calculated pair: (2, 4)
INFO:root:Calculated pair: (2, 5)
INFO:root:Calculated pair: (2, 6)
INFO:root:Calculated pair: (2, 7)
INFO:root:Calculated pair: (2, 8)
INFO:root:Calculated pair: (2, 9)
INFO:root:Calculated pair: (3, 3)
INFO:root:Calculated pair: (3, 4)
INFO:root:Calc

Matriz de covarianzas
 
[[ 4.05203526e-06 -1.52714431e-05  1.48719448e-07 -1.41620203e-04
   1.84728434e-05  4.87494436e-04 -3.51167285e-06]
 [-1.52714431e-05  2.27453744e-03  3.62620995e-05  2.20027251e-02
  -2.71442341e-03 -4.37493046e-02  3.85668146e-04]
 [ 1.48719448e-07  3.62620995e-05  6.31606116e-07  3.45510310e-04
  -4.48514137e-05 -6.67996554e-04  5.96165845e-06]
 [-1.43304808e-05  3.48638305e-03  5.06073795e-05  3.00187154e+11
   7.80677370e+10 -3.64890547e+11  2.85718936e+09]
 [ 5.17081585e-05 -7.53050845e-03 -1.21516893e-04  7.80662760e+10
   2.03021929e+10 -9.48929551e+10  7.43036902e+08]
 [ 2.10005227e-04 -1.04705249e-02 -1.47759188e-04 -2.17943507e+11
  -5.66781013e+10 -4.17008586e+12  3.24878782e+10]
 [-1.55363700e-06  1.27036013e-04  1.89814176e-06  1.69793197e+09
   4.41561952e+08  3.24878782e+10 -2.53103237e+08]]


In [6]:
def comparison(created, expected):
    return 100*np.abs(1 - (created/expected))

In [7]:
print('ERRORES')

print('Error Omega_m: ' + str(np.sqrt(Cov[0, 0])))
print('Error h: ' + str(np.sqrt(Cov[1, 1])))
print('Error Omega_b: ' + str(np.sqrt(Cov[2, 2])))
print('Error ns: ' + str(np.sqrt(Cov[3, 3])))
print('Error sigma8: ' + str(np.sqrt(Cov[4, 4])))
print('Error Aia: ' + str(np.sqrt(Cov[5, 5])))
print('Error Cia: ' + str(np.sqrt(Cov[6, 6])))

print(' ')

# Relative errors

print('RELATIVE ERRORS')

err_Om_m = (np.sqrt(Cov[0, 0]) / Omega_m0_fid)
err_h = np.sqrt(Cov[1, 1]) / h_fid
err_Om_b = np.sqrt(Cov[2, 2]) / Omega_b0_fid
err_ns = np.sqrt(Cov[3, 3]) / ns_fid
err_sig = np.sqrt(Cov[4, 4]) / sigma8_fid
err_A = np.sqrt(Cov[5, 5]) / Aia
err_C = np.sqrt(Cov[6, 6]) / Cia

print('Error Relativo Omega_m: ' + str(err_Om_m))
print('Error Relativo h: ' + str(err_h))
print('Error Relativo Omega_b: ' + str(err_Om_b))
print('Error Relativo ns: ' + str(err_ns))
print('Error Relativo sigma8: ' + str(err_sig))
print('Error Relativo A: ' + str(err_A))
print('Error Relativo C: ' + str(err_C))

print(' ')

print('COMPARACION')
## Comparison with the paper (the numbers must be under 10)

S_Om_m = 0.018
S_h = 0.21
S_Om_b = 0.47
S_ns = 0.035
S_sig = 0.0087

print('Comparison for Omega_m: ' + str(comparison(err_Om_m, S_Om_m)))
print('Comparison for h: ' + str(comparison(err_h, S_h)))
print('Comparison for Omega_b: ' + str(comparison(err_Om_b, S_Om_b)))
print('Comparison for ns: ' + str(comparison(err_ns, S_ns)))
print('Comparison for sigma8: ' + str(comparison(err_sig, S_sig)))

ERRORES
Error Omega_m: 0.0020129667795793066
Error h: 0.0476921108995855
Error Omega_b: 0.0007947365074398209
Error ns: 547893.3783812895
Error sigma8: 142485.76381705928
Error Aia: nan
Error Cia: nan
 
RELATIVE ERRORS
Error Relativo Omega_m: 0.006290521186185333
Error Relativo h: 0.0711822550740082
Error Relativo Omega_b: 0.01589473014879642
Error Relativo ns: 570722.2691471766
Error Relativo sigma8: 174614.90663855305
Error Relativo A: nan
Error Relativo C: nan
 
COMPARACION
Comparison for Omega_m: 65.05266007674815
Comparison for h: 66.10368805999609
Comparison for Omega_b: 96.61814252153268
Comparison for ns: 1630634954.7062187
Comparison for sigma8: 2007067792.3971615


/var/folders/86/5my1j53j5bn4j010w8s9jjjc0000gn/T/ipykernel_2632/1024903501.py:8: RuntimeWarning: invalid value encountered in sqrt
  print('Error Aia: ' + str(np.sqrt(Cov[5, 5])))
/var/folders/86/5my1j53j5bn4j010w8s9jjjc0000gn/T/ipykernel_2632/1024903501.py:9: RuntimeWarning: invalid value encountered in sqrt
  print('Error Cia: ' + str(np.sqrt(Cov[6, 6])))
/var/folders/86/5my1j53j5bn4j010w8s9jjjc0000gn/T/ipykernel_2632/1024903501.py:22: RuntimeWarning: invalid value encountered in sqrt
  err_A = np.sqrt(Cov[5, 5]) / Aia
/var/folders/86/5my1j53j5bn4j010w8s9jjjc0000gn/T/ipykernel_2632/1024903501.py:23: RuntimeWarning: invalid value encountered in sqrt
  err_C = np.sqrt(Cov[6, 6]) / Cia
